In [ ]:
import pandas as pd
import numpy as np
import calendar

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.corpus import cess_esp

from nltk import UnigramTagger, BigramTagger, TrigramTagger #N-Grams para analizar contextos
from nltk.tag.hmm import HiddenMarkovModelTagger

#from nltk.chunk.

from wordcloud import WordCloud, STOPWORDS
import re

import spacy
import matplotlib.pyplot as plt
import es_core_news_sm
import en_core_web_sm


import urllib.request
from bs4 import BeautifulSoup

import warnings


warnings.filterwarnings("ignore")

c:\Users\danie\anaconda3\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'es_core_news_sm' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.3.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
#En teoría no es necesario, pero especifico el idioma por si acaso
#nlp=spacy.load('es_core_news_sm')
nlp=spacy.load('en_core_web_sm')
#npl=en_core_web_sm.load()

In [ ]:
#Cargo el archivo csv con los tweets y genero un dataframe
df_Madrid=pd.read_csv(r"C:\Users\danie\Desktop\Apuntes\M5-Inteligencia de negocio y visualización\Práctica final\tweets_Madrid_en.csv")

In [ ]:
df_Madrid

,geo,text,user,user_location
0,NaN,Had dreams of travelling to Madrid via (sleepe...,Fergus_Walsh,Newcastle-upon-Tyne
1,NaN,RT @DannyLast: Just dug out another photo I to...,antouninho,Al di là. A volte al di sopra.
2,NaN,RT @RSVPMagazine: 🇪🇸 Spanish airline Iberia ha...,Jason_Moore_,Ireland
3,NaN,@pattyannlowe Just read an article about Irela...,marc06rules,NaN
4,NaN,RT @AIRCOP_ROPAN: The group @AIRCOP_UNODC GETA...,LuisYBurgos,NaN
...,...,...,...,...
326,NaN,@CPVIndia I have a layover of 18 hours at Jedd...,Sanjay20201985,NaN
327,NaN,@KSAmofaEN I have a layover of 18 hours at Jed...,Sanjay20201985,NaN
328,NaN,@KSAMOFA I have a layover of 18 hours at Jedda...,Sanjay20201985,NaN
329,NaN,Madrid Airport heading for Porto. 6 hours of d...,Antialiasfactry,Hamburg


In [ ]:
#Copio el contenido de los tweets en una lista y lo transformo en una cadena de texto
texto = []

for i in df_Madrid['text']:
    texto.append(i)
str_texto = "".join(texto)
test = "".join(texto) #lo necesitaré para utilizar HiddenMarkovModelTagger 

In [ ]:
#Elimino emojis y símbolos
def Emojify(str_texto):
    regrex_pattern = re.compile(pattern ="["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',str_texto)

In [ ]:
str_texto=Emojify(str_texto)

In [ ]:
#Elimino urls
pattern = re.compile(r'(https?://)?(www\.)?(\w+\.)?(\w+)(\.\w+)(/.+)?')
str_texto = re.sub(pattern,'',str_texto)

In [ ]:
#Los pronobres y determinantes no me interesan, luego me quedo únicamente con los nombres

nombres=[doc.text for doc in nlp(str_texto) if doc.is_stop!=True and doc.is_punct!=True and doc.pos_=='NOUN']

str_nombres=" ".join(nombres)
print(str_nombres)

dreams train time frame hotels u airport flight majo aviation aircraft airplane sunset love thankyou crewlife airportview bed COVIDisAirborne airport travell hours immigration airport queues picture citizen arrivals citizen arrivals Photos Photos flight BA2835 Photos airport picture citizen arrivals citizen arrivals Photos airport picture citizen arrivals citizen arrivals Photos welcome daughter marble floor airport flight @ClaraJeffery day testing airport L1011 credit President aircraft time suite Como schmancy family member citizen passport agency citizen passport L1011 credit ft ⁦@CAGNE_GATWICK⁩ ⁦ deals hotels welcome flight way welcome flight way welcome @emirates family nonsto Art madrid turismo liveforthestory hours flight case option plane hour flight gate Occasion Art madrid turismo welcome addition places sabotage welcome flight way welcome information situation parents tomorrow @Ragonar_Photo plane flight way welcome news today .@Iberia_en flight way welcome @frazierapproves 

In [ ]:
#Función para extraer los tokens:

def gen_tokens(_texto):
    tokens=word_tokenize(_texto,"english")
    tokens=[word.lower() for word in tokens if word.isalpha()]
    
    clean_tokens=tokens[:]
    
    for tok in tokens:
        if tok in stopwords.words('english'):
            clean_tokens.remove(tok)
        
    lem_tokens = []
    separator=' '
    
    for tok in nlp(separator.join(clean_tokens)):
        lem_tokens.append(tok.lemma_)
        
    return lem_tokens

In [ ]:
#creo una tabla igual a un dataframe:

def text_density(_tokens):
    tabla=pd.DataFrame(columns=['Palabra','Recuento','Densidad'])
    
    freq =nltk.FreqDist(_tokens)
    palabras=len(_tokens)
    
    for key, val in freq.items():
        densidad=val/palabras
        row=pd.Series([key, val, densidad], index=tabla.columns)
        tabla=tabla.append(row, ignore_index=True)
        pd.concat([tabla,row],ignore_index=False)
        
    return tabla.sort_values('Densidad',ascending=True)

In [ ]:
tabla_densidad=text_density(gen_tokens(str_nombres))

In [ ]:
tabla_densidad.sort_values(by=['Densidad'], inplace=True, ascending=False)

tabla_densidad

,Palabra,Recuento,Densidad
7,flight,12,0.090226
27,welcome,11,0.082707
24,citizen,8,0.06015
6,airport,8,0.06015
45,way,7,0.052632
...,...,...,...
68,teleport,1,0.007519
28,daughter,1,0.007519
13,love,1,0.007519
29,marble,1,0.007519


In [ ]:
#descardo un excell con la tabla de densidades de Madrid
tabla_densidad.to_excel(r'C:\Users\danie\Desktop\Apuntes\M5-Inteligencia de negocio y visualización\Práctica final\topics_Madrid_en.xlsx', index = False)

In [ ]:
#Cargo el archivo csv con los tweets y genero un dataframe
df_Barcelona=pd.read_csv(r"C:\Users\danie\Desktop\Apuntes\M5-Inteligencia de negocio y visualización\Práctica final\tweets_Barcelona_en.csv")

In [ ]:
df_Barcelona

,geo,text,user,user_location
0,NaN,RT @Rob_Kimbell: Irish airline Ryanair to conn...,16Suse,NaN
1,NaN,Bus from Bilbao Airport to San Sebastian https...,_Barcelona__,NaN
2,NaN,@DrHaroldNews Whizzed through Barcelona Airpor...,WillyWonka079,NaN
3,NaN,RT @Rob_Kimbell: Irish airline Ryanair to conn...,annh_me,Seaside
4,NaN,Is there a single person left in Dublin or is ...,poflaherty98,NaN
...,...,...,...,...
495,NaN,"RT @lowtheband: Man, they gotta pay their bagg...",Dubbelmono,NaN
496,NaN,"RT @lowtheband: Man, they gotta pay their bagg...",ggarenfeld,Tilburg
497,NaN,"RT @lowtheband: Man, they gotta pay their bagg...",moonsocket,"Nova Scotia, Canada"
498,NaN,Trains from Barcelona Airport to the City http...,_Barcelona__,NaN


In [ ]:
#Copio el contenido de los tweets en una lista y lo transformo en una cadena de texto
texto = []

for i in df_Barcelona['text']:
    texto.append(i)
str_texto = "".join(texto)
test = "".join(texto) #lo necesitaré para utilizar HiddenMarkovModelTagger 

In [ ]:
#Elimino emojis y símbolos
def Emojify(str_texto):
    regrex_pattern = re.compile(pattern ="["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',str_texto)

In [ ]:
str_texto=Emojify(str_texto)

In [ ]:
#Elimino urls
pattern = re.compile(r'(https?://)?(www\.)?(\w+\.)?(\w+)(\.\w+)(/.+)?')
str_texto = re.sub(pattern,'',str_texto)

In [ ]:
#Los pronobres y determinantes no me interesan, luego me quedo únicamente con los nombres

nombres=[doc.text for doc in nlp(str_texto) if doc.is_stop!=True and doc.is_punct!=True and doc.pos_=='NOUN']

str_nombres=" ".join(nombres)

In [ ]:
#Función para extraer los tokens:

def gen_tokens(_texto):
    tokens=word_tokenize(_texto,"english")
    tokens=[word.lower() for word in tokens if word.isalpha()]
    
    clean_tokens=tokens[:]
    
    for tok in tokens:
        if tok in stopwords.words('english'):
            clean_tokens.remove(tok)
        
    lem_tokens = []
    separator=' '
    
    for tok in nlp(separator.join(clean_tokens)):
        lem_tokens.append(tok.lemma_)
        
    return lem_tokens

In [ ]:
#creo una tabla igual a un dataframe:

def text_density(_tokens):
    tabla=pd.DataFrame(columns=['Palabra','Recuento','Densidad'])
    
    freq =nltk.FreqDist(_tokens)
    palabras=len(_tokens)
    
    for key, val in freq.items():
        densidad=val/palabras
        row=pd.Series([key, val, densidad], index=tabla.columns)
        tabla=tabla.append(row, ignore_index=True)
        pd.concat([tabla,row],ignore_index=False)
        
    return tabla.sort_values('Densidad',ascending=True)

In [ ]:
tabla_densidad=text_density(gen_tokens(str_nombres))

In [ ]:
tabla_densidad.sort_values(by=['Densidad'], inplace=True, ascending=False)

tabla_densidad

,Palabra,Recuento,Densidad
10,airport,47,0.109049
100,centre,32,0.074246
99,makeup,19,0.044084
20,flight,18,0.041763
103,charter,15,0.034803
...,...,...,...
110,date,1,0.00232
51,chef,1,0.00232
113,dreamcatcherinspainrt,1,0.00232
114,ddiddirere,1,0.00232


In [ ]:
#descardo un excell con la tabla de densidades de Madrid
tabla_densidad.to_excel(r'C:\Users\danie\Desktop\Apuntes\M5-Inteligencia de negocio y visualización\Práctica final\topics_Barcelona_en.xlsx', index = False)